In [1]:
import pandas as pd

In [2]:
npr = pd.read_csv('npr.csv')

In [3]:
npr.head()

,Article
0,"In the Washington of 2016, even when the polic..."
1,Donald Trump has used Twitter — his prefe...
2,Donald Trump is unabashedly praising Russian...
3,"Updated at 2:50 p. m. ET, Russian President Vl..."
4,"From photography, illustration and video, to d..."


In [4]:
npr['Article'][0]

'In the Washington of 2016, even when the policy can be bipartisan, the politics cannot. And in that sense, this year shows little sign of ending on Dec. 31. When President Obama moved to sanction Russia over its alleged interference in the U. S. election just concluded, some Republicans who had long called for similar or more severe measures could scarcely bring themselves to approve. House Speaker Paul Ryan called the Obama measures ”appropriate” but also ”overdue” and ”a prime example of this administration’s ineffective foreign policy that has left America weaker in the eyes of the world.” Other GOP leaders sounded much the same theme. ”[We have] been urging President Obama for years to take strong action to deter Russia’s worldwide aggression, including its   operations,” wrote Rep. Devin Nunes,  . chairman of the House Intelligence Committee. ”Now with just a few weeks left in office, the president has suddenly decided that some stronger measures are indeed warranted.” Appearing 

In [5]:
len(npr)

11992

## Data preprocessing

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
cv = CountVectorizer(max_df=0.9, min_df=2, stop_words="english")
#                        |           |            |__ discard english stop words (ex : a, the, at, for, of, ...) 
#                        |           |_____ discard words that shows up less than 2 documents (mostly misspelled words)
#                        |_________________ discard the words that shows 90% of documents (very common words)

In [9]:
dtm = cv.fit_transform(npr['Article'])
# dtm : document term matrix

In [10]:
dtm # 11992 ( number of documents ) x 54777 ( number of turms )

<11992x54777 sparse matrix of type '<class 'numpy.int64'>'
	with 3033388 stored elements in Compressed Sparse Row format>

## Latent dirichlet allocation

In [11]:
from sklearn.decomposition import LatentDirichletAllocation

In [12]:
LDA = LatentDirichletAllocation(n_components=7, random_state=42)
#                                   |
#                               number of topics

In [13]:
LDA.fit(dtm)

LatentDirichletAllocation(n_components=7, random_state=42)

## grab the vocabulary of words

In [14]:
len(cv.get_feature_names())

54777

In [16]:
type(cv.get_feature_names())

list

In [25]:
import random

for i in range(10):
    print(cv.get_feature_names()[random.randint(0,54777)])

# words took from doc

stivers
liang
wreaking
georgi
crystallizing
hitchhiker
broaching
inaccuracies
hunnam
outfits


## Grab the topics

In [28]:
LDA.components_.shape

(7, 54777)

In [27]:
type(LDA.components_) # contain probabilities of each word

numpy.ndarray

In [29]:
LDA.components_

array([[8.64332806e+00, 2.38014333e+03, 1.42900522e-01, ...,
        1.43006821e-01, 1.42902042e-01, 1.42861626e-01],
       [2.76191749e+01, 5.36394437e+02, 1.42857148e-01, ...,
        1.42861973e-01, 1.42857147e-01, 1.42906875e-01],
       [7.22783888e+00, 8.24033986e+02, 1.42857148e-01, ...,
        6.14236247e+00, 2.14061364e+00, 1.42923753e-01],
       ...,
       [3.11488651e+00, 3.50409655e+02, 1.42857147e-01, ...,
        1.42859912e-01, 1.42857146e-01, 1.42866614e-01],
       [4.61486388e+01, 5.14408600e+01, 3.14281373e+00, ...,
        1.43107628e-01, 1.43902481e-01, 2.14271779e+00],
       [4.93991422e-01, 4.18841042e+02, 1.42857151e-01, ...,
        1.42857146e-01, 1.43760101e-01, 1.42866201e-01]])

In [31]:
single_topic = LDA.components_[0]

In [34]:
single_topic.argsort() # return index positions of sorted array (least to greatest)

array([ 2475, 18302, 35285, ..., 22673, 42561, 42993], dtype=int64)

In [35]:
single_topic.argsort()[-10:] # last 10 elements (greatest 10 or 10 highest probability words)

array([33390, 36310, 21228, 10425, 31464,  8149, 36283, 22673, 42561,
       42993], dtype=int64)

In [38]:
top_twenty_words = single_topic.argsort()[-20:]

In [39]:
for i in top_twenty_words:
    print(cv.get_feature_names()[i])

president
state
tax
insurance
trump
companies
money
year
federal
000
new
percent
government
company
million
care
people
health
said
says


## Grab the highest probability words per topics

In [40]:
for index,topic in enumerate(LDA.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['companies', 'money', 'year', 'federal', '000', 'new', 'percent', 'government', 'company', 'million', 'care', 'people', 'health', 'said', 'says']


THE TOP 15 WORDS FOR TOPIC #1
['military', 'house', 'security', 'russia', 'government', 'npr', 'reports', 'says', 'news', 'people', 'told', 'police', 'president', 'trump', 'said']


THE TOP 15 WORDS FOR TOPIC #2
['way', 'world', 'family', 'home', 'day', 'time', 'water', 'city', 'new', 'years', 'food', 'just', 'people', 'like', 'says']


THE TOP 15 WORDS FOR TOPIC #3
['time', 'new', 'don', 'years', 'medical', 'disease', 'patients', 'just', 'children', 'study', 'like', 'women', 'health', 'people', 'says']


THE TOP 15 WORDS FOR TOPIC #4
['voters', 'vote', 'election', 'party', 'new', 'obama', 'court', 'republican', 'campaign', 'people', 'state', 'president', 'clinton', 'said', 'trump']


THE TOP 15 WORDS FOR TOPIC #5
['years', 'going', 've', 'life', 'don', 'new', 'way', 'music', 'really', 'time', 'know', 'think',

## Attach topic numbers to original articles

In [41]:
topic_results = LDA.transform(dtm)

In [42]:
topic_results.shape

(11992, 7)

In [44]:
topic_results[0].round(3) # probability of document belonging to topic

array([0.016, 0.683, 0.   , 0.   , 0.3  , 0.   , 0.   ])

In [45]:
topic_results[0].argmax() # index of highest

1

In [46]:
npr["topic"] = topic_results.argmax(axis=1)

In [47]:
npr

,Article,topic
0,"In the Washington of 2016, even when the polic...",1
1,Donald Trump has used Twitter — his prefe...,1
2,Donald Trump is unabashedly praising Russian...,1
3,"Updated at 2:50 p. m. ET, Russian President Vl...",1
4,"From photography, illustration and video, to d...",2
...,...,...
11987,The number of law enforcement officers shot an...,1
11988,"Trump is busy these days with victory tours,...",4
11989,It’s always interesting for the Goats and Soda...,3
11990,The election of Donald Trump was a surprise to...,4
